Michael Sexton - Intro to AI project


Project Writeup


Here is my take on the Mancala AI implementation using Minimax and Alpha-Beta Minimax algorithms. My code operates by setting up a Mancala game board with the proper necessary functions in place to play a game with variable pits_per_player and stones_per_pit. In addition, I have a random move generator which plays comepletly randomly that can be used to play against as well. For the project, I coded a minimax algorithm and several necessary functions such as utility_function and possible_actions which allow the minimax algorithm to work as expected and maintain the code's readability. I used the random and copy frameworks only, choosing not to use the aima-python library since minimax is already implemented in it, and I wanted to learn how to create it myself.

For testing, once I had my minimax functions working properly, I ran a loop that simulated a number of games and found how many times the minimax algorithm would win versus a completely randomly generated move player. I found that my minimax algorithm beat the randomly generated move player 90% of the time after 25 simulated games. This number fluctuates but its usually between 85% and 100% of the time. In addition, I examined how long it took to actually run the code. With the minimax algorithm and the alpha-beta minimax algorithm bearing the same results of a win percentage, I found that the alpha-beta minimax algorithm ran in 1/3 the time it took the normal minimax algorithm to run. This shows how useful the pruning of the alpha-beta algorithm actually is when used in a large game tree. I also altered the depth of the algorithm and the size of the mancala board to see what the affects where of the algorithm. Increasing the depth makes the sim take more time to run, but has a higher win percentage as it can see more moves ahead, and decreasing the size of the game board makes the sim better by decreasing the total number of possible game states that can occur.

In conclusion, I learned a lot about how effective an artificial agent can be when playing a game, and this has opened the door to many possible projects in which implementing a similar agent can vastly increase my odds. Overall, the functions weren't too difficult to implement when given the algorithm in the book, it just took some time to set up how I would actually arrange my array of game states and how to utilize the already created Mancala functions.

In [9]:
import random
import copy
from random import randint

In [7]:
class Mancala:
    def __init__(self, pits_per_player=3, stones_per_pit = 2):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        current_player =  self.current_player
        p1_pits_index = self.p1_pits_index
        p2_pits_index = self.p2_pits_index
        if current_player == 1:
            # check if pit is out of bounds or empty
            if (pit > self.pits_per_player or pit < 1) or self.board[pit-1] == 0:
                
                return False
        elif current_player == 2:
            if (pit > self.pits_per_player or pit < 1) or self.board[pit+self.pits_per_player] == 0:
                
                return False
        return True
        pass

        
    def random_move_generator(self):
        while True:
            pit = randint(1, self.pits_per_player)
            if self.valid_move(pit):
                
                return pit

        
    
    
    def play(self,pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        # is it valid?
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_mancala = self.board[self.p2_mancala_index]
        
        valid_truth_value = self.valid_move(pit)
        if not valid_truth_value:
            return self.board
        # check win conditon
        self.winning_eval()
        #if valid, append to moves
        if (valid_truth_value):
            self.moves.append((self.current_player, pit))
        
       
        
        if self.current_player == 1:
            pit = pit-1
            skipped_mancala = self.p2_mancala_index
            scoring_mancala = self.p1_mancala_index
            mancala_index_low = self.p1_pits_index[0]
            mancala_index_high = self.p1_pits_index[1]
        else:
            pit = pit+self.pits_per_player
            skipped_mancala = self.p1_mancala_index
            scoring_mancala = self.p2_mancala_index
            mancala_index_low = self.p2_pits_index[0]
            mancala_index_high = self.p2_pits_index[1]
        # count the number of stones in the pit
        stones_in_selected_pit = self.board[pit]
        # set the selected pit to 0
        self.board[pit] = 0
        # loop through the next pit, dropping 1 stone each time until i hit 0
        while (stones_in_selected_pit > 0):  
            pit = pit+1
            if (pit == skipped_mancala):
                pit = pit+1
                if (pit == ((self.pits_per_player*2)+2)):
                    pit = 0
            if (pit == ((self.pits_per_player*2)+2)):
                pit = 0
            
            if pit >= mancala_index_low and pit <= mancala_index_high and self.board[pit] == 0 and stones_in_selected_pit == 1:
                
                opposite_pit = scoring_mancala - (pit - mancala_index_low)
                self.board[scoring_mancala] += self.board[opposite_pit] + 1
                self.board[opposite_pit] = 0
                self.board[pit] = 0


                # swap player
                if (self.current_player == 1):
                    self.current_player = 2
                else:
                    self.current_player = 1
                return self.board
            self.board[pit] = self.board[pit] + 1
            
            stones_in_selected_pit = stones_in_selected_pit - 1
            
        # swap player
        if (self.current_player == 1):
            self.current_player = 2
        else:
            self.current_player = 1
            
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        p1_pits_index = self.p1_pits_index
        p2_pits_index = self.p2_pits_index
        
        p1_mancala_index = self.p1_mancala_index
        p2_mancala_index = self.p2_mancala_index
        
        i = p1_pits_index[0]
        j = p2_pits_index[0]
        
        p1_stone_count = 0
        p2_stone_count = 0
        
        # loop through p1 and p2 side to see if empty
        while (i <= p1_pits_index[1]):
            p1_stone_count += self.board[i]
            i = i+1
        while (j <= p2_pits_index[1]):
            p2_stone_count += self.board[j]
            j = j+1
        # if empty print game over and winner
        if p1_stone_count == 0 or p2_stone_count == 0:
            
            p1_mancala_stones = self.board[p1_mancala_index]
            p2_mancala_stones = self.board[p2_mancala_index]

            # print statement for single term plays, this does not play nice with the loop for x number of random vs random game sims
            # if (game.board[game.p1_mancala_index]>game.board[game.p2_mancala_index]):
            #      print("GAME OVER! - WINNER PLAYER 1")
            # elif (game.board[game.p1_mancala_index]<game.board[game.p2_mancala_index]):
            #     print("GAME OVER! - WINNER PLAYER 2")
            # else:
            #     print("Tie game!")
            
            return True  # Game ends

        else:
            return False
        
        pass
    
    def utility_function(self):
        # find the utility of a given state for a given player
        
        p1_mancala = self.board[self.p1_mancala_index]
        p2_mancala = self.board[self.p2_mancala_index]
        return (p1_mancala - p2_mancala) if self.current_player == 1 else (p2_mancala - p1_mancala)

        
    def possible_actions(self):
        #creates an array of all possible moves from a given state
        current_player = self.current_player
        actions = []
        
        if current_player == 1:
            mancala_index_low = self.p1_pits_index[0]
            mancala_index_high = self.p1_pits_index[1]
        else:
            mancala_index_low = self.p2_pits_index[0]
            mancala_index_high = self.p2_pits_index[1]
            
        for pit in range(mancala_index_low, mancala_index_high + 1): 
            
            
            if(self.board[pit] > 0):  
                actions.append(pit)
        
        return actions
            
    def minimax_search(self, depth = 5):
        
        #set a copy of the game to run the algo on that is the current board and current player and create a copy
        copied_game = copy.deepcopy(self)
        
        # Get the best move from max_value on the copy
        _, move = copied_game.max_value(depth)
        # the +1 is because of indexing miss match since the entire search is done in 0 based, then the actual board wants 1-based.
        return move+1
        
    
        
        
    def max_value(self, depth):
        
        #base case, where the winning move was found or max depth was hit
        if self.winning_eval() or depth == 0:
            
            return self.utility_function(), None
        # initializing v and best_move
        v = float('-inf')
        best_move = None
        
        #loop where for every possible action in possible_actions for a given state
        for action in self.possible_actions():
            
            # try possible action
            new_state = self.result(action)
            
            # now simulate that the opposing player will take the best possible move for themselves or min_value
            v2, _ = new_state.min_value(depth-1)
            
            # find the greatest minimum value from all possible states and set it
            if v2 > v:
                
                v, best_move = v2, action
               
            # continue looping until we find the maximum value of the possible moves and return it
        return v, best_move
        
    def min_value(self, depth):
        
        #base case
        if self.winning_eval() or depth == 0:
            
            return self.utility_function(), None
        
        #initializing v and best_move
        v = float('inf')
        best_move = None
        
        # for every possible action in possible_actions for a given state
        for action in self.possible_actions():
            # try possible action
            
            new_state = self.result(action)
            # now simulate that the opposing player will take the best possible move for themselves or max_value
            
            v2, _ = new_state.max_value(depth-1)
            
            # find the lowest maximium value from all possible states
            if v2 < v:
                
                v, best_move = v2, action
                
                # continue looping until we find the minimum value of the possible moves and return it
        return v, best_move
      
    def result(self, action):
        #take current board and create a copy where i execute the move i pass in
        current_player = self.current_player
        new_board = copy.deepcopy(self)
        
        # Simulate the action on a copy of the board
        
        new_board.play(action+1)
        
        if (new_board.current_player == 1):
            new_board.current_player = 2
        else:
            new_board.current_player = 1
        return new_board

    def alpha_beta_search(self, depth = 5):
        #set a copy of the game to run the algo on that is the current board and current player and create a copy
        copied_game = copy.deepcopy(self)
        alpha = float('-inf')
        beta = float('inf')
        # Get the best move from max_value on the copy
        _, move = copied_game.max_value_AB(depth, alpha, beta)
        return move+1
        
        
    def max_value_AB(self, depth, alpha, beta):
        
        #base case, where the winning move was found or max depth was hit
        if self.winning_eval() or depth == 0:
            
            return self.utility_function(), None
        # initializing v and best_move
        v = float('-inf')
        best_move = None
        
        #loop where for every possible action in possible_actions for a given state
        for action in self.possible_actions():
            # try possible action
            
            new_state = self.result(action)
            
            # now simulate that the opposing player will take the best possible move for themselves or min_value
            v2, _ = new_state.min_value_AB(depth-1,alpha,beta)
            
            # find the greatest minimum value from all possible states and set it
            if v2 > v:
                
                v, best_move = v2, action
                alpha = max(alpha,v)
            if v >= beta: 
                return v, best_move
            # continue looping until we find the maximum value of the possible moves and return it
        return v, best_move
        
    def min_value_AB(self, depth, alpha, beta):
        
        #base case
        if self.winning_eval() or depth == 0:
            
            return self.utility_function(), None
        
        #initializing v and best_move
        v = float('inf')
        best_move = None
        
        # for every possible action in possible_actions for a given state
        for action in self.possible_actions():
            # try possible action
            
            new_state = self.result(action)
            # now simulate that the opposing player will take the best possible move for themselves or max_value
            
            v2, _ = new_state.max_value_AB(depth-1,alpha,beta)
            
            # find the lowest maximium value from all possible states
            if v2 < v:
                
                v, best_move = v2, action
                beta = min(beta,v)
            if v <= alpha:
                return v, best_move
                # continue looping until we find the minimum value of the possible moves and return it
        return v, best_move
      

In [50]:
# Mancala part 1 
game = Mancala()
game.display_board()
# Player 1 selects pit 1 (1-based index)
game.play(1)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 3
game.play(3)
game.display_board()

# Player 2 selects pit 2
game.play(2)
game.display_board()

# Player 1 selects pit 1
game.play(1)
game.display_board()

# Printing the list of moves
print("\nList of valid moves:")
for move in game.moves:
    player, pit = move
    print(f"Player {player} selected pit {pit}")


P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 2 | 2 | <- 2
3 -> |_2_|_2_| <- 1
         0         
Turn: P1
P1               P2
     ____0____     
1 -> | 0 | 2 | <- 3
2 -> | 3 | 2 | <- 2
3 -> |_3_|_2_| <- 1
         0         
Turn: P2
P1               P2
     ____1____     
1 -> | 0 | 3 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_3_|_2_| <- 1
         0         
Turn: P1
P1               P2
     ____1____     
1 -> | 0 | 3 | <- 3
2 -> | 3 | 1 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P2
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P1
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 3 | 0 | <- 2
3 -> |_0_|_3_| <- 1
         1         
Turn: P1

List of valid moves:
Player 1 selected pit 1
Player 2 selected pit 2
Player 1 selected pit 3
Player 2 selected pit 2


In [122]:
#game init
game = Mancala(pits_per_player=6, stones_per_pit = 4)
game.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1


In [123]:
#real player turn 1
game.play(2)
game.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 4 | <- 5
3 -> | 5 | 4 | <- 4
4 -> | 5 | 4 | <- 3
5 -> | 5 | 4 | <- 2
6 -> |_5_|_4_| <- 1
         0         
Turn: P2


In [124]:
#random computer turn 1
move = game.random_move_generator()
game.play(move)
game.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 5 | <- 3
5 -> | 5 | 5 | <- 2
6 -> |_5_|_0_| <- 1
         0         
Turn: P1


In [125]:
# real-user turn 2
game.play(6)
game.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 6 | <- 4
4 -> | 5 | 6 | <- 3
5 -> | 5 | 6 | <- 2
6 -> |_0_|_1_| <- 1
         1         
Turn: P2


In [126]:
# computer (random) turn 2
move = game.random_move_generator()
game.play(move)
game.display_board()

P1               P2
     ____1____     
1 -> | 5 | 5 | <- 6
2 -> | 1 | 6 | <- 5
3 -> | 5 | 7 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 6 | <- 2
6 -> |_0_|_1_| <- 1
         1         
Turn: P1


In [127]:
# real-user turn 3
game.play(2)
game.display_board()

P1               P2
     ____1____     
1 -> | 5 | 5 | <- 6
2 -> | 0 | 6 | <- 5
3 -> | 6 | 7 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 6 | <- 2
6 -> |_0_|_1_| <- 1
         1         
Turn: P2


In [128]:
# computer (random) turn 3
move = game.random_move_generator()
game.play(move)
game.display_board()

P1               P2
     ____1____     
1 -> | 5 | 5 | <- 6
2 -> | 0 | 6 | <- 5
3 -> | 6 | 7 | <- 4
4 -> | 5 | 0 | <- 3
5 -> | 5 | 7 | <- 2
6 -> |_0_|_0_| <- 1
         1         
Turn: P1


In [129]:
# real-user turn 4
game.play(5)
game.display_board()

P1               P2
     ____1____     
1 -> | 5 | 5 | <- 6
2 -> | 0 | 6 | <- 5
3 -> | 6 | 7 | <- 4
4 -> | 5 | 1 | <- 3
5 -> | 0 | 8 | <- 2
6 -> |_1_|_1_| <- 1
         2         
Turn: P2


In [130]:
# computer (random) turn 4
move = game.random_move_generator()
game.play(move)
game.display_board()

P1               P2
     ____2____     
1 -> | 6 | 6 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 7 | 7 | <- 4
4 -> | 6 | 1 | <- 3
5 -> | 0 | 8 | <- 2
6 -> |_1_|_1_| <- 1
         2         
Turn: P1


In [131]:
# real-user turn 5
game.play(3)
game.display_board()

P1               P2
     ____2____     
1 -> | 6 | 6 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 0 | 7 | <- 4
4 -> | 7 | 2 | <- 3
5 -> | 1 | 9 | <- 2
6 -> |_2_|_2_| <- 1
         3         
Turn: P2


In [132]:
# computer (random) turn 5
move = game.random_move_generator()
game.play(move)
game.display_board()

P1               P2
     ____2____     
1 -> | 6 | 6 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 0 | 7 | <- 4
4 -> | 7 | 3 | <- 3
5 -> | 1 | 10 | <- 2
6 -> |_2_|_0_| <- 1
         3         
Turn: P1


In [17]:
# RANDOM VS RANDOM 10000 game sims

Player_1_Win_Count = 0
Player_2_Win_Count = 0
Tie_Count = 0
Turn_Count = 0
games_to_sim = 10000
for game_number in range(1, games_to_sim):
    game = Mancala(pits_per_player=6, stones_per_pit = 4)
    game.current_player = 1
    
    while not game.winning_eval(): 
        
        
        if game.current_player == 1:
            move = game.random_move_generator()
            game.play(move)
        else:
            move = game.random_move_generator()
            game.play(move)
        
        Turn_Count += 1
    if (game.board[game.p1_mancala_index]>game.board[game.p2_mancala_index]):
        Player_1_Win_Count += 1
    elif (game.board[game.p1_mancala_index]<game.board[game.p2_mancala_index]):
        Player_2_Win_Count += 1
    else:
        Tie_Count += 1
    
    
print("After",games_to_sim,"simulations, on average, player 1 wins" , (Player_1_Win_Count/(Player_2_Win_Count+Player_1_Win_Count)), "percentage of the time.")
print("On average it takes this many turns to complete a game:" , (Turn_Count/2)/games_to_sim)
print("Tie count", Tie_Count)

After 10000 simulations, on average, player 1 wins 0.5118961038961038 percentage of the time.
On average it takes this many turns to complete a game: 21.9193
Tie count 374


In [16]:
#MINIMAX VS RANDOM 25 game sims
Player_1_Win_Count = 0
Player_2_Win_Count = 0
Tie_Count = 0
Turn_Count = 0
games_to_sim = 25
for game_number in range(1, games_to_sim):
    game = Mancala(pits_per_player=6, stones_per_pit = 4)
    game.current_player = 1
    
    while not game.winning_eval(): 
        
        if game.current_player == 1:
            move = game.minimax_search(depth = 5)
            game.play(move)
        else:
            move = game.random_move_generator()
            game.play(move)
        
        Turn_Count += 1
    if (game.board[game.p1_mancala_index]>game.board[game.p2_mancala_index]):
        Player_1_Win_Count += 1
    elif (game.board[game.p1_mancala_index]<game.board[game.p2_mancala_index]):
        Player_2_Win_Count += 1
    else:
        Tie_Count += 1
    
    
print("After",games_to_sim,"simulations, on average, player 1 wins" , (Player_1_Win_Count/(Player_2_Win_Count+Player_1_Win_Count)), "percentage of the time.")
print("On average it takes this many moves to complete a game:" , (Turn_Count/2)/games_to_sim)
print("Tie count", Tie_Count)

After 25 simulations, on average, player 1 wins 0.9090909090909091 percentage of the time.
On average it takes this many moves to complete a game: 16.84
Tie count 2


In [28]:
#ALPHA BETA MINIMAX VS RANDOM 25 game sims
Player_1_Win_Count = 0
Player_2_Win_Count = 0
Tie_Count = 0
Turn_Count = 0
games_to_sim = 100
for game_number in range(1, games_to_sim):
    game = Mancala(pits_per_player=6, stones_per_pit = 4)
    game.current_player = 1
    
    while not game.winning_eval(): 
        
        if game.current_player == 1:
            move = game.alpha_beta_search(depth = 2)
            game.play(move)
        else:
            move = game.random_move_generator()
            game.play(move)
        
        Turn_Count += 1
    if (game.board[game.p1_mancala_index]>game.board[game.p2_mancala_index]):
        Player_1_Win_Count += 1
    elif (game.board[game.p1_mancala_index]<game.board[game.p2_mancala_index]):
        Player_2_Win_Count += 1
    else:
        Tie_Count += 1
    
    
print("After",games_to_sim,"simulations, on average, player 1 wins" , (Player_1_Win_Count/(Player_2_Win_Count+Player_1_Win_Count)), "percentage of the time.")
print("On average it takes this many moves to complete a game:" , (Turn_Count/2)/games_to_sim)
print("Tie count", Tie_Count)


After 100 simulations, on average, player 1 wins 0.9270833333333334 percentage of the time.
On average it takes this many moves to complete a game: 15.105
Tie count 3
